# Estimating Trip Mode Choice

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "trip_mode_choice"

from activitysim.estimation.larch import component_model
model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/trip_mode_choice/trip_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/trip_mode_choice/trip_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/trip_mode_choice/trip_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/trip_mode_choice/trip_mode_choice_values_combined.parquet


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_one,1.0000,T
coef_nest_root,1.0000,T
coef_nest_AUTO,0.7200,T
coef_nest_AUTO_DRIVEALONE,0.3500,T
coef_nest_AUTO_SHAREDRIDE2,0.3500,T
...,...,...
coef_walk_transit_ASC_walk_eatout,1.3504,F
walk_express_penalty,10.0000,T
adjust_tnc_shared,30.0000,T


#### Utility specification

In [5]:
data.spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_if_didnt_drive...,DRIVEALONEFREE - Unavailable if didn't drive t...,is_atwork_subtour & ~work_tour_is_SOV,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,util_Walk_not_available_for_long_distances,Walk not available for long distances,@df.tour_mode_is_walk & (od_skims['DISTWALK'] ...,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,util_Bike_not_available_for_long_distances,Bike not available for long distances,@df.tour_mode_is_walk & (od_skims['DISTBIKE'] ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,util_origin_density_index,Origin density index,@origin_density_applied*(origin_density_index_...,NaN,NaN,NaN,NaN,NaN,NaN,coef_ivt,...,coef_ivt,coef_ivt,NaN,NaN,NaN,NaN,NaN,NaN,coef_ivt,coef_ivt
378,util_walk_express_penalty,Walk-express penalty for intermediate stops,@walk_express_penalty * ~(df.first_trip | df.f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Chooser data

In [6]:
data.chooser_data

,trip_id,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,...,drive_heavyrail_available_outbound,drive_heavyrail_available_inbound,drive_commuter_available_outbound,drive_commuter_available_inbound,walk_ferry_available,drive_ferry_available,distance,distance_walk_od,distance_bike_od,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
1870,14961,WALK,WALK,0.0,0.0,0.0,0.0,0.0,10.52,10.63286,...,False,False,False,False,False,False,3.78,3.78,3.78,7
1870,14965,WALK,WALK,0.0,0.0,0.0,0.0,0.0,10.31,10.63286,...,False,False,False,False,False,False,3.79,3.79,3.79,7
20468,163745,WALK_LOC,WALK_LOC,0.0,0.0,0.0,0.0,0.0,4.72,6.62312,...,True,False,True,False,False,False,2.07,2.07,2.07,9
20468,163749,WALK_LOC,TNC_SINGLE,0.0,0.0,0.0,0.0,0.0,4.72,6.62312,...,False,True,False,True,False,False,2.07,2.07,2.07,20
27055,216441,SHARED3FREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,6.77,4.26534,...,False,False,False,False,False,False,2.49,2.49,2.49,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310202384,2481619077,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,2.74,1.91196,...,False,False,False,False,False,False,1.66,2.55,2.55,1
310212634,2481701073,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,6.42,2.66128,...,False,False,False,False,False,False,2.57,2.57,2.57,1
310212634,2481701077,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,6.42,2.66128,...,False,False,False,False,False,False,2.57,2.57,2.57,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.doctor(repair_ch_av='-')
model.loglike()

-94787.94601063678

In [8]:
model.maximize_loglike(method='SLSQP', options={"maxiter": 1000})

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999,-999.000000,-999.000000,-999.0000,-999.0,-999.0,0.0,1
1,1.000000,1.000000,1.0000,1.0,1.0,0.0,1
coef_age010_trn,0.131028,0.131028,0.0000,-inf,inf,0.0,0
coef_age1619_da,0.432972,0.432972,0.0000,-inf,inf,0.0,0
coef_age16p_sr,-0.403317,-0.403317,0.0000,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
coef_walk_transit_ASC_walk_school,-1.566510,-1.566510,-1.1828,-inf,inf,0.0,0
coef_walk_transit_ASC_walk_shopping,-1.250219,-1.250219,-0.1943,-inf,inf,0.0,0
coef_walk_transit_ASC_walk_social,-0.846190,-0.846190,-0.7651,-inf,inf,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/optimization.py:338: UserWarning: SLSQP may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )


┣                x: -999                                  -999.000000
┃                   1                                        1.000000
┃                   coef_age010_trn                          0.131028
┃                   coef_age1619_da                          0.432972
┃                   coef_age16p_sr                          -0.403317
┃                                                             ...    
┃                   coef_walk_transit_ASC_walk_school       -1.566510
┃                   coef_walk_transit_ASC_walk_shopping     -1.250219
┃                   coef_walk_transit_ASC_walk_social       -0.846190
┃                   coef_walk_transit_ASC_walk_univ         -1.031100
┃                   coef_walk_transit_ASC_walk_work          0.296352
┃                   Length: 272, dtype: float64
┣          logloss: 0.5159237004389526
┣        d_logloss: -999                                   0.000000
┃                   1                                      0.000000
┃                   coef_age010_trn                       -0.000081
┃                   coef_age1619_da                       -0.000052
┃                   coef_age16p_sr                        -0.000041
┃                                                            ...   
┃                   coef_walk_transit_ASC_walk_school      0.000097
┃                   coef_walk_transit_ASC_walk_shopping    0.000045
┃                   coef_walk_transit_ASC_walk_social     -0.000075
┃                   coef_walk_transit_ASC_walk_univ        0.000000
┃                   coef_walk_transit_ASC_walk_work       -0.000163
┃                   Length: 272, dtype: float64
┣              nit: 304
┣             nfev: 317
┣             njev: 304
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=212, microseconds=168906)
┣           method: 'SLSQP'
┣          n_cases: 162587
┣ iteration_number: 304
┣          loglike: -83882.486683268

### Estimated coefficients

In [9]:
model.parameter_summary()

,Value,Null Value
Parameter,,
-999,-999.,0.00
1,1.00,0.00
coef_age010_trn,0.131,0.00
coef_age1619_da,0.433,0.00
coef_age16p_sr,-0.403,0.00
coef_bike_ASC_rh,-7.00,0.00
coef_bike_ASC_walk_eatout,-1.69,0.00
coef_bike_ASC_walk_escort,-13.5,0.00
coef_bike_ASC_walk_othdiscr,-1.32,0.00


# Output Estimation Results

In [10]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [11]:
for m in model:
    m.to_xlsx(
        result_dir/f"{m.title}_{modelname}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [12]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_one,1.000000,T
1,coef_nest_root,1.000000,T
2,coef_nest_AUTO,0.720000,T
3,coef_nest_AUTO_DRIVEALONE,0.350000,T
4,coef_nest_AUTO_SHAREDRIDE2,0.350000,T
...,...,...,...
309,coef_walk_transit_ASC_walk_eatout,1.418767,F
310,walk_express_penalty,10.000000,T
311,adjust_tnc_shared,30.000000,T
312,coef_origin_density_applied_work_univ_school,0.000000,T
